In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget

import numpy as nps
import matplotlib.pyplot as plt
import ibl_tools
from ibl_tools.core_files import ExperimentNames, ExperimentFiles
from ibl_tools.plot import visualize_frames

# Read all experiments
experiment = ExperimentNames()
# Select subset of experiments for demo
experiment_names = ['b8875d4d-e28f-4976-859e-bf8a5dd20024_paws_small',
 'cf7972dd-e408-4759-a3e2-e404a71f13ee_paws_small',
 '735c72f4-1b43-4eb6-8754-f2e2705cdcca_paws_small',
 'b1c80c88-4e4b-4651-966c-10d3eaf9b99e_paws_small',
 'fafe6343-d1d5-4979-814b-6024e97ffb89_paws_small']

In [2]:
# Extract data of a subject
ii = 2
subject_id = experiment_names[ii]

In [3]:
# Read all filenames for subject
exp_files = ExperimentFiles(subject_id, ii)

/run/user/1000/gvfs/smb-share:server=locker-smb.engram.rc.zi.columbia.edu,share=paninski-locker,user=ADCU%5Cekb2154/data/ibl/dlc-networks


In [4]:
# Load traces
(train_, val_, test_) = exp_files.load_trace_features()
(train_data, train_slice) = train_
(val_data, val_slice) = val_
(test_data, test_slice) = test_

In [7]:
from behdata.pyhsmm.models import fit_ar_pyhsmm_models
from behdata.utils_model import create_schedule
import os
import time
import pickle
import numpy as np

In [8]:
# Create jobs for file
schedule_id = 0

if schedule_id == 0:
    schedule = create_schedule(
        {
            "model": ["arhmm"],
            "emissions": ["ar"],
            "transitions": ["sticky"],
            "lags": 1,
            "affine": True,
            "alpha": 10,
            "kappa": 10**np.arange(2, 6),#10**np.arange(2, 11)[::2],
            "K": range(2, 12),  # number of discrete states
            "N_iters": 2000,
            "seed": range(1),
        }
    )

Created schedule containing 40 configurations.


In [9]:
log_dir = "/data/model_01_out/right_paw/subject_{:02}/".format(ii)
if not os.path.isdir(log_dir):
    os.makedirs(log_dir)

In [10]:
print("Number of Experiments to run {}".format(len(schedule)))
#%%

for config_id, config in enumerate(schedule):

    print("\n--------------------------------------------\n\n")
    print("Running experiment {} out of {}\n".format(config_id, len(schedule)))

    fname = "--".join([f"{key}-{value}" for key, value in config.items()])

    # Check if file exists then pass
    file_out = os.path.join(log_dir, fname + ".pkl")

    if os.path.isfile(file_out):
        print("\n Skipping file:\n{}\n already exists!\n".format(file_out))
        continue

    start = time.time()
    model = config["model"]

    try:

        if model == "arhmm":

            model, lls_train, lls_val, lls_test, _, _ = fit_ar_pyhsmm_models(
                train_data,
                val_data,
                test_data,
                K=config["K"],
                N_iters=config["N_iters"],
                seed=config["seed"],
                lags=config["lags"],
                affine=config["affine"],
                alpha=config["alpha"],
                kappa=config["kappa"],
                observations=config["emissions"]
            )
            # Construct dictionary of data
            
            d = {"model": model,
                 "lls_train": lls_train,
                 "lls_val":lls_val,
                 "lls_test":lls_test
                }

        print("Run for {} \n".format(time.time() - start))

        # Write
        print("Writting:\n{}\n".format(file_out))
        file_handler = open(file_out, "wb")
        pickle.dump(d, file_handler)
        file_handler.close()
        print("Wrote file in {} \n".format(time.time() - start))

    except Exception as e:
        print("\n*******\n")
        print("\nCrashed for config: \n%s\n" % str(config))
        print(e)
        print("\n*******\n")


Number of Experiments to run 40

--------------------------------------------


Running experiment 0 out of 40


 Skipping file:
/data/model_01_out/right_paw/subject_02/model-arhmm--emissions-ar--transitions-sticky--lags-1--affine-True--alpha-10--kappa-100--K-2--N_iters-2000--seed-0.pkl
 already exists!


--------------------------------------------


Running experiment 1 out of 40


 Skipping file:
/data/model_01_out/right_paw/subject_02/model-arhmm--emissions-ar--transitions-sticky--lags-1--affine-True--alpha-10--kappa-100--K-3--N_iters-2000--seed-0.pkl
 already exists!


--------------------------------------------


Running experiment 2 out of 40


 Skipping file:
/data/model_01_out/right_paw/subject_02/model-arhmm--emissions-ar--transitions-sticky--lags-1--affine-True--alpha-10--kappa-100--K-4--N_iters-2000--seed-0.pkl
 already exists!


--------------------------------------------


Running experiment 3 out of 40


 Skipping file:
/data/model_01_out/right_paw/subject_02/model-arh